Recommendation de films basé sur le genre , les thèmes et le vote


In [1]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.preprocessing import MinMaxScaler
import numpy as np
import pandas as pd


Charger les données grace à pandas

In [2]:
donnees = pd.read_csv("dataset movies imdb\TMDB_movie_dataset_v11.zip")
donnees.head()
#donnees = donnees.fillna('Unknow')

<>:1: SyntaxWarning: invalid escape sequence '\T'
<>:1: SyntaxWarning: invalid escape sequence '\T'
C:\Users\Freud Arthur\AppData\Local\Temp\ipykernel_32544\2693940860.py:1: SyntaxWarning: invalid escape sequence '\T'
  donnees = pd.read_csv("dataset movies imdb\TMDB_movie_dataset_v11.zip")


,id,title,vote_average,vote_count,status,release_date,revenue,runtime,adult,backdrop_path,...,original_title,overview,popularity,poster_path,tagline,genres,production_companies,production_countries,spoken_languages,keywords
0,27205,Inception,8.364,34495,Released,2010-07-15,825532764,148,False,/8ZTVqvKDQ8emSGUEMjsS4yHAwrp.jpg,...,Inception,"Cobb, a skilled thief who commits corporate es...",83.952,/oYuLEt3zVCKq57qu2F8dT7NIa6f.jpg,Your mind is the scene of the crime.,"Action, Science Fiction, Adventure","Legendary Pictures, Syncopy, Warner Bros. Pict...","United Kingdom, United States of America","English, French, Japanese, Swahili","rescue, mission, dream, airplane, paris, franc..."
1,157336,Interstellar,8.417,32571,Released,2014-11-05,701729206,169,False,/pbrkL804c8yAv3zBZR4QPEafpAR.jpg,...,Interstellar,The adventures of a group of explorers who mak...,140.241,/gEU2QniE6E77NI6lCU6MxlNBvIx.jpg,Mankind was born on Earth. It was never meant ...,"Adventure, Drama, Science Fiction","Legendary Pictures, Syncopy, Lynda Obst Produc...","United Kingdom, United States of America",English,"rescue, future, spacecraft, race against time,..."
2,155,The Dark Knight,8.512,30619,Released,2008-07-16,1004558444,152,False,/nMKdUUepR0i5zn0y1T4CsSB5chy.jpg,...,The Dark Knight,Batman raises the stakes in his war on crime. ...,130.643,/qJ2tW6WMUDux911r6m7haRef0WH.jpg,Welcome to a world without rules.,"Drama, Action, Crime, Thriller","DC Comics, Legendary Pictures, Syncopy, Isobel...","United Kingdom, United States of America","English, Mandarin","joker, sadism, chaos, secret identity, crime f..."
3,19995,Avatar,7.573,29815,Released,2009-12-15,2923706026,162,False,/vL5LR6WdxWPjLPFRLe133jXWsh5.jpg,...,Avatar,"In the 22nd century, a paraplegic Marine is di...",79.932,/kyeqWdyUXW608qlYkRqosgbbJyK.jpg,Enter the world of Pandora.,"Action, Adventure, Fantasy, Science Fiction","Dune Entertainment, Lightstorm Entertainment, ...","United States of America, United Kingdom","English, Spanish","future, society, culture clash, space travel, ..."
4,24428,The Avengers,7.710,29166,Released,2012-04-25,1518815515,143,False,/9BBTo63ANSmhC4e6r62OJFuK2GL.jpg,...,The Avengers,When an unexpected enemy emerges and threatens...,98.082,/RYMX2wcKCBAr24UyPD7xwmjaTn.jpg,Some assembly required.,"Science Fiction, Action, Adventure",Marvel Studios,United States of America,"English, Hindi, Russian","new york city, superhero, shield, based on com..."


In [6]:
from sqlalchemy import create_engine
engine = create_engine('sqlite:///data.sqlite')

donnees.to_sql('Users', engine, if_exists='replace', index=False)

1268303

In [3]:
print(f" Les colonnes du dataset sont {donnees.columns}")


 Les colonnes du dataset sont Index(['id', 'title', 'vote_average', 'vote_count', 'status', 'release_date',
       'revenue', 'runtime', 'adult', 'backdrop_path', 'budget', 'homepage',
       'imdb_id', 'original_language', 'original_title', 'overview',
       'popularity', 'poster_path', 'tagline', 'genres',
       'production_companies', 'production_countries', 'spoken_languages',
       'keywords'],
      dtype='object')


In [99]:
donnees[['title' , 'vote_average']].loc[donnees.vote_average == 0]
donnees.vote_average.value_counts()

vote_average
0.000     914118
6.000      30860
5.000      29992
10.000     26973
7.000      24797
           ...  
1.917          1
1.875          1
9.875          1
1.625          1
0.875          1
Name: count, Length: 5025, dtype: int64

In [43]:
donnees.loc[: , ['genres' ,'keywords']] = donnees[['genres' ,'keywords']].fillna('Unknow')

Le scaler et la vectorisation

In [44]:

vectorizer = TfidfVectorizer()
donnees['combined'] = donnees['genres'] + " " + donnees['keywords']

# genre_and_keywords = donnees[['genres', 'keywords']]
donnees_vectoriser = vectorizer.fit_transform(donnees['combined'])
votes = donnees[['vote_average']]



In [106]:
def suggestions_de_films(film):
   

    data = donnees.loc[donnees['title'].str.lower() == film.lower() , ['genres' , 'keywords']]
    if data.empty : 
        print ("Aucun film trouvé")
        return

    print(f"Correspondance pour le film {film}")
    print(data)

    recommendation =[]

    data_found = data.iloc[0]
    texte_film = [data_found['genres'] + " " + data_found['keywords']]
    print(texte_film)
    genre_film_vectoriser = vectorizer.transform(texte_film)

    #for index , f in enumerate(genre_and_keywords_vectoriser):
    score  = cosine_similarity( genre_film_vectoriser , donnees_vectoriser  ).flatten()
    
        
    recommendation = list(zip(donnees['title'] , score, donnees['vote_average']))

    recommendation.sort(key=lambda x: x[1], reverse=True)

    return recommendation[1:6]


        

In [27]:
film = donnees.loc[donnees['title'] == 'Hulk' , ['genres', 'keywords']].iloc[0]
film

genres                     Science Fiction, Adventure, Action
keywords    california, monster, general, mutation, san fr...
Name: 756, dtype: object

In [7]:

data = donnees.loc[donnees['title'] == 'Iron Man' , ['genres' , 'keywords'] ]
print(data)
donnees['title'] == 'Iron Man'

                                     genres  \
12       Action, Science Fiction, Adventure   
82111                                 Drama   
95068                                 Drama   
276586                    Action, Adventure   
497224                               Unknow   
687552   Action, Animation, Science Fiction   
1129146                              Unknow   

                                                  keywords  
12       middle east, superhero, arms dealer, malibu, b...  
82111                        pre-code, gold digger, boxing  
95068                                    sports, film noir  
276586                                        martial arts  
497224                                              Unknow  
687552                                              Unknow  
1129146                                             Unknow  


0          False
1          False
2          False
3          False
4          False
           ...  
1268298    False
1268299    False
1268300    False
1268301    False
1268302    False
Name: title, Length: 1268303, dtype: bool

In [107]:
suggestions_de_films("Iron Man")




Correspondance pour le film Iron Man
                                     genres  \
12       Action, Science Fiction, Adventure   
82111                                 Drama   
95068                                 Drama   
276586                    Action, Adventure   
497224                               Unknow   
687552   Action, Animation, Science Fiction   
1129146                              Unknow   

                                                  keywords  
12       middle east, superhero, arms dealer, malibu, b...  
82111                        pre-code, gold digger, boxing  
95068                                    sports, film noir  
276586                                        martial arts  
497224                                              Unknow  
687552                                              Unknow  
1129146                                             Unknow  
['Action, Science Fiction, Adventure middle east, superhero, arms dealer, malibu, based on comic, 

[('Iron Man 2', np.float64(0.7013185877155945), 6.835),
 ('Black Panther', np.float64(0.6773933801688972), 7.39),
 ('Fantastic Four', np.float64(0.6624860439817399), 0.0),
 ('Armor Wars', np.float64(0.6541866295082038), 0.0),
 ('Inhumans: The First Chapter', np.float64(0.6479299811457198), 5.297)]

In [9]:
suggestions_de_films("title")

Correspondance pour le film title
        genres keywords
589104  Unknow   Unknow
751174  Unknow   Unknow
940030  Unknow   Unknow
['Unknow Unknow']


[('Return', np.float64(1.0), 6.223),
 ('Barely Legal Baby Fat', np.float64(1.0), 6.792),
 ('Anplagghed al cinema', np.float64(1.0), 7.0),
 ('Return', np.float64(1.0), 6.237),
 ('The Right of Youth', np.float64(1.0), 6.587)]

In [10]:
b = donnees['title'] == 'Interstellar'
b

0          False
1           True
2          False
3          False
4          False
           ...  
1268298    False
1268299    False
1268300    False
1268301    False
1268302    False
Name: title, Length: 1268303, dtype: bool

* Fonction pour verifier la similarité entre le film qu'un tuilisateur a tapé celui et ceux qui se retrouvent dans la base 
* Agir sur un autre dataset

In [132]:
donnee_title = donnees.title.copy()

In [133]:
donnee_title.drop_duplicates(inplace=True)

In [134]:
donnee_title = donnee_title.dropna()

In [136]:
# Créer un vectoriser sur les lignes de mon dataset

title_vectoriser = TfidfVectorizer()
title_sp =title_vectoriser.fit_transform(donnee_title)

In [137]:
def suggestions_de_titres(title):
    search = [title]
    search_v = title_vectoriser.transform(search)

    score_title = cosine_similarity(title_sp , search_v ).flatten()
    results = list(zip(donnee_title , score_title))

    results.sort(key=lambda x : x[1] , reverse=True)

    print(results[:5])
    return results[:5]

In [139]:
suggestions_de_titres('Naruto')

[('Naruto', np.float64(1.0)), ('Naruto 20th Anniversary - Road of Naruto', np.float64(0.8377359941726894)), ('Secret of Naruto', np.float64(0.8197679868147817)), ('The Last: Naruto the Movie', np.float64(0.724259666361851)), ('Naruto x UT', np.float64(0.6917410527365556))]


[('Naruto', np.float64(1.0)),
 ('Naruto 20th Anniversary - Road of Naruto', np.float64(0.8377359941726894)),
 ('Secret of Naruto', np.float64(0.8197679868147817)),
 ('The Last: Naruto the Movie', np.float64(0.724259666361851)),
 ('Naruto x UT', np.float64(0.6917410527365556))]

In [13]:
donnees.groupby('genres')['vote_average'].count()
#donnees.sort_values(by='vote_average', ascending=False).head(10)

genres
Action                                                          8317
Action, Action                                                     1
Action, Action, Crime                                              1
Action, Action, Thriller                                           1
Action, Adventure                                                976
                                                                ... 
Western, War, Adventure, History, Science Fiction, Animation       1
Western, War, Drama                                                2
Western, War, Drama, Music                                         1
Western, War, Romance                                              1
Western, War, Thriller                                             1
Name: vote_average, Length: 14138, dtype: int64

In [16]:
corpus = [
    'This is the first document.',
    'This document is the second document.',
    'And this is the third one.',
    'Is this the first document?',
]

vectorizer = TfidfVectorizer()
X = vectorizer.fit_transform(corpus)
print(vectorizer.get_feature_names_out())
print(X.toarray())

['and' 'document' 'first' 'is' 'one' 'second' 'the' 'third' 'this']
[[0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]
 [0.         0.6876236  0.         0.28108867 0.         0.53864762
  0.28108867 0.         0.28108867]
 [0.51184851 0.         0.         0.26710379 0.51184851 0.
  0.26710379 0.51184851 0.26710379]
 [0.         0.46979139 0.58028582 0.38408524 0.         0.
  0.38408524 0.         0.38408524]]


In [ ]:
import pandas as pd
import requests
import zipfile
import io
# URL of the raw zip file on GitHub
url = "https://github.com/FreudArthur/Movie-Recommandation/raw/main/Recommandation%20imdb/dataset%20movies%20imdb/TMDB_movie_dataset_v11.zip"

# Download the zip file content
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Read the content as bytes
    zip_content = io.BytesIO(response.content)

    # Open the zip file
    with zipfile.ZipFile(zip_content, 'r') as zip_ref:
        # Assuming the CSV file inside the zip has a specific name,
        # you might need to adjust this if the name is different.
        # You can list the contents of the zip using zip_ref.namelist()
        # For example, to list contents:
        # print(zip_ref.namelist())
        csv_file_name = 'TMDB_movie_dataset_v11.csv' # Replace with the actual CSV file name

        # Extract and read the CSV file
        try:
            with zip_ref.open(csv_file_name) as csv_file:
                data = pd.read_csv(csv_file)
                display(data.head())
        except KeyError:
            print(f"Error: '{csv_file_name}' not found in the zip file.")
            print("Available files in the zip:", zip_ref.namelist())

else:
    print(f"Error: Could not download the file. Status code: {response.status_code}")

In [ ]:
corpus = ['Bonjour' , 'Salur commrnt tu vas' , 'Maman je veux rentrer a la maison', 'Maman ou est tu' , 'Bonjour bro' , 'La police est ouverte' , 'Ma maman et mon papa' ]

v= TfidfVectorizer()
v.fit(corpus)
sp = v.transform(corpus)
print(sp.todense())

print(v.get_feature_names_out())
sp.shape

cosine_similarity(sp[3] , sp[2])

[[1.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.        ]
 [0.         0.         0.52064676 0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.         0.         0.52064676 0.43218152
  0.52064676 0.        ]
 [0.         0.         0.         0.         0.         0.43884665
  0.3642804  0.         0.43884665 0.31137481 0.         0.
  0.         0.         0.         0.43884665 0.         0.
  0.         0.43884665]
 [0.         0.         0.         0.48900396 0.         0.
  0.         0.         0.         0.41798437 0.         0.58910044
  0.         0.         0.         0.         0.         0.48900396
  0.         0.        ]
 [0.63870855 0.76944876 0.         0.         0.         0.
  0.         0.         0.         0.         0.         0.
  0.         0.         0.  

array([[0.13014981]])

### Gérer l'optimisation et le préchargement pour la mise en ligne

* Charger par chunks au lieu de tout charger en une seule fois 100000
(for chunk in pd.read_csv ("gros_dataset.csv", chunksize=chunksize):
    filtered = chunk[chunk['colonne'] == 'valeur']
    results.append(filtered))

* Modifier le type de valeur pour faciliter (dtype_dict = {
    'id': 'int32',
    'category': 'category',
    'score': 'float32',
  
})

* Selectionner uniquement les colonnes importantes cad titre vote average keywords genres overviews

* Effectuer les calculs et les vectorisations juste apres et les stocker dans une nouvelle colonne (df = pd.read_csv("gros_dataset.csv", usecols=colonnes_utiles)
)
